<a href="https://colab.research.google.com/github/avdlaan/dissertation/blob/main/ktrain_summarize_using_ROUGE_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip3 install ktrain==0.28.2
!pip install tensorflow==2.4.1
!pip3 install rouge/requirements.txt
!pip3 install rouge-score
!pip install ntlk

Using Ktrain from an example Jupyter notebook

In [ ]:
import numpy as np
import pandas as pd 
import glob
import json
import math

The data is hard coded in CSV format, the next step is to extract it from a corpus using pandas or other text extract mechanism

In [ ]:
from ktrain.text.summarization import TransformerSummarizer
ts = TransformerSummarizer()

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Extract data from the corpus first using pandas and data frame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

all_json = glob.glob('/content/drive/MyDrive/train/document_parses/json_test/*.json', recursive=True)

# let us get the list of articles
len(all_json)
print(len(all_json))

Mounted at /content/drive
39


In [ ]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:400]}... {self.body_text[:500]}...'

In [ ]:
first_row = FileReader(all_json[0])
print(first_row)

000a0fc8bbef80410199e690191dc3076a290117: Background: Various transcription factors are involved in the process of mutually exclusive expression and clonal variation of the Plasmodium multigene (var) family. Recent studies revealed that a P. falciparum SWI/SNF-related matrix-associated actin-dependent regulator of chromatin (PfSWIB) might trigger stage-specific programmed cell death (PCD), and was not only crucial for the survival and dev... Plasmodium falciparum, which causes malignant malaria such as cerebral malaria (CM) or pregnancy-associated malaria (PAM) [1, 2] , is the cause of death to 435,000 individuals annually [3] . Plasmodium falciparum erythrocyte membrane protein 1 (PfEMP1), which is encoded by the var gene family comprising approximately 60 members, is the major virulence factor involved in the antigenic variation and clinical pathogenicity of falciparum malaria [4, 5] . It is remarkable that the expression of var f...


In [ ]:
# enable the scorer
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)


dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'summary':[], 'score':[]}
for idx, entry in enumerate(all_json):
    try:
        content = FileReader(entry)
        if (idx % 1000 == 0):
            print(f"processing {idx} of {len(all_json)} ")
        dict_['paper_id'].append(content.paper_id)
        dict_['abstract'].append(content.abstract)
        dict_['body_text'].append(content.body_text)
        summary_document = ts.summarize(content.body_text)
        dict_['summary'].append(summary_document)
        scores = scorer.score(content.abstract,
                      summary_document)
        dict_['score'].append(scores)
    except:
        pass
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text','summary','score'])
#df_covid = pd.DataFrame(dict_, columns=['body_text'])
#docs = df_covid['body_text'].values.tolist()
#docs = df_covid.values.tolist()
df_covid.head(39)
#print(docs)

processing 0 of 39 


,paper_id,abstract,body_text,summary,score
0,000a0fc8bbef80410199e690191dc3076a290117,Background: Various transcription factors are ...,"Plasmodium falciparum, which causes malignant ...",Plasmodium falciparum causes malignant malaria...,"{'rouge1': (0.5, 0.06371191135734072, 0.113022..."
1,00a0df94d685622b8a7894f6a952e97a7e89ccf6,"Citation: Kostyuk, S.V.; Proskurnina, E.V.; Er...",Recovery of muscle tissue damaged by diseases ...,Recovery of muscle tissue damaged by diseases ...,"{'rouge1': (0.14285714285714285, 0.14516129032..."
2,00a00d0edc750db4a0c299dd1ec0c6871f5a4f24,,This is an Open Access article distributed und...,The Middle East Respiratory Syndrome (MERS) cr...,"{'rouge1': (0.0, 0.0, 0.0), 'rougeL': (0, 0, 0)}"
3,00a1a921b93d9773f46d21ac22b1363371c7d535,Efforts to mitigate the global spread of the s...,a1111111111 a1111111111 a1111111111 a111111111...,The global public has been inundated with info...,"{'rouge1': (0.4788732394366197, 0.395348837209..."
4,00a0ab182dc01b6c2e737dfae585f050dcf9a7a5,,The world was made aware of a newly discovered...,The world was made aware of a newly discovered...,"{'rouge1': (0.0, 0.0, 0.0), 'rougeL': (0, 0, 0)}"
5,00a0c02539f89317beb2954f72b20fa91dd0cc87,,"On 23 March 2020, the UK announced the ""stay a...","The UK announced the ""stay at home"" order and ...","{'rouge1': (0.0, 0.0, 0.0), 'rougeL': (0, 0, 0)}"
6,0a0c7465fb4d6dfc7c5c3b4d019bf12a9a3cea6b,The spread of coronavirus disease infection ac...,The global pandemic caused by the coronavirus ...,The global pandemic caused by the coronavirus ...,"{'rouge1': (0.5333333333333333, 0.189349112426..."
7,0a1a319ffd87d5b69e7738e57c382a3fc3c8b420,,"In December 2019, the severe acute respiratory...","In December 2019, the severe acute respiratory...","{'rouge1': (0.0, 0.0, 0.0), 'rougeL': (0, 0, 0)}"
8,0a00fe902ce5f523ceca3d01dc68dd1ddaac253b,We propose the Modified Corona Score (MCorona ...,"On behalf of all authors, the corresponding au...","After the December 2019 outbreak in China, the...","{'rouge1': (0.46, 0.16428571428571428, 0.24210..."
9,0a0d9d7f7ccd750820cbe07415db17e4d8ae2f2d,The emergence of a novel strain of betacoronav...,To respond effectively to the recent Severe Ac...,CoV-AbDab is a database that documents molecul...,"{'rouge1': (0.6222222222222222, 0.133333333333..."


In [ ]:
df_covid.to_csv('data2.csv',sep='|',header=True)
!cp data.csv "drive/My Drive/"


# import BLEU
#from nltk.translate.bleu_score import sentence_bleu

#import ROUGE
